# Game Tree Search

We start with defining the abstract class `Game`, for turn-taking *n*-player games. We rely on, but do not define yet, the concept of a `state` of the game; we'll see later how individual games define states. For now, all we require is that a state has a `state.to_move` attribute, which gives the name of the player whose turn it is. ("Name" will be something like `'X'` or `'O'` for tic-tac-toe.) 

We also define `play_game`, which takes a game and a dictionary of  `{player_name: strategy_function}` pairs, and plays out the game, on each turn checking `state.to_move` to see whose turn it is, and then getting the strategy function for that player and applying it to the game and the state to get a move.

# These are previous codes from games4e. The class game builds interface that is used by class TicTacToe and Board. The TicTacToc class builds the moves and players logic. The Board class creates all the possible states(9 possible moves) of the TicTacToe game class. The player interface allows each player to use an AI alogoritm or user move. 

In [ ]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools 
import time
import sys
import re
import unittest
cache = functools.lru_cache(10**6)

In [ ]:
class Game:
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `utility`. You will also need to set the .initial attribute to the 
    initial state; this can be done in the constructor."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
    
    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError
        

def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose: 
            print('Player', player, 'move:', move)
            print("\nCurrent Tic-Tac-Toe Board")
            print(state)
    return state

# Minimax-Based Game Search Algorithms

We will define several game search algorithms. Each takes two inputs, the game we are playing and the current state of the game, and returns a a `(value, move)` pair, where `value` is the utility that the algorithm computes for the player whose turn it is to move, and `move` is the move itself.

First we define `minimax_search`, which exhaustively searches the game tree to find an optimal move (assuming both players play optimally), and `alphabeta_search`, which does the same computation, but prunes parts of the tree that could not possibly have an affect on the optimnal move.  

# These are minimax and alpha-beta pruning that will be used in play_game() method. These are the two main alogithms in the tic-tac-toe game. Both expected inputs are TicTacToe() class and possible states that are recusivlely updated after each player's moves. The expected output are highest moves of a player.

In [ ]:
def minimax_search(game, state):
    """Search game tree to determine best move; return (value, move) pair."""

    player = state.to_move

    def max_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v, move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v, move = v2, a

        return v, move

    return max_value(state)

infinity = math.inf

def alphabeta_search(game, state):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

# A Simple Game: Tic-Tac-Toe

We have the notion of an abstract game, we have some search functions; now it is time to define a real game; a simple one, tic-tac-toe. Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(2, 2)` is the bottom right (on a board of size `height=width=3`).

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an `height` by `width` board, needing `k` in a row to win.
    'X' plays first against 'O'."""
    

    def __init__(self, height=3, width=3, k=3):
        self.k = k # k in a row
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)

    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = (0 if not win else +1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board): print(board)
    


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

States in tic-tac-toe (and other games) will be represented as a `Board`, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes: 
- `.to_move` to name the player whose move it is; 
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- possibly other attributes, as specified by keywords. 

As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `off` otherwise. The class has a `__hash__` method, so instances can be stored in hash tables.

In [ ]:
class Board(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

# Players

We need an interface for players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below:

In [ ]:
import re

def random_player(game, state): 
    return random.choice(list(game.actions(state)))


def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]


# Menu User game: I added a funcitonality ( menuSystem() ) that loads up a menu for the user to play against a specific alogorithm. Based on user input, the menu will load and display the specific output. It also allows user to see how fast each alogithm are against each other. The expected input is that a player chooses 1-7. These numbers represents who the players play against or if they want the alogorithms to peform against each other. The expected output is that the menuSystem() will display the game and allow the user to play the game. The user() allows the user to play against an AI, menu() displays the menu, menuInput() takes in 1-7 choice, and menuSystem() creates the menuSystem with these indiviual parts

In [ ]:
#User player function - allows user to pick a move to play against a specified player
def user(game, state):
    #displays the current state of board everytime after a player puts a move on board
    print("\nCurrent Tic-Tac-Toe Board")
    game.display(state)
    print("")
    #displays the possible moves to user after both player puts a move on board
    print("Possible moves You can make: " + str((game.actions(state))))
    print("Enter in (0, 1) format")
    print("")
    #intializes user move
    yourMove = None
    #displays all possible moves if is available or not empty, else display no moves left
    if game.actions(state) != None and len(game.actions(state)) != 0:
        move_string = ""
        #try-catch to catch errors when entering move input if user doesn't have correct format to enter
        try:
            #if move doesn't follow format, a random move will be picked for user, else user move will be valid
            move_string = input('Your turn X: ')
            if not (re.match(r'^\([0-9]{1}, [0-9]{1}\)$', move_string) or re.match(r'^\([0-9]{1},[0-9]{1}\)$', move_string) or re.match(r'[0-9]{1},[0-9]{1}$', move_string) or re.match(r'[0-9]{1}, [0-9]{1}$', move_string)):
                move_string = ""
                randomCh = random.choice(list(game.actions(state)))
                print("You made move : " + randomCh)
                yourMove = eval(randomCh)
            else:
                 yourMove = eval(move_string)
        #possbile list of errors it can catch and display the error message and error class
        except (TypeError, NameError, SyntaxError) as err:
            print(type(err), err)
            yourMove = move_string
            print("Unvalid nubmer")
            pass
    else:
        print("No moves remaining")
    #returns individual user moves
    return yourMove

  
#create the menu component at beginning of menu system    
def menu():
    print("\n******Welcome to Tic-Tac-Toe Game.*******")
    print("Menu")
    print("1. User VS Minimax")
    print("2. User VS Alpha-Beta")
    print("3. User vs random player")
    print("4. random player vs Minimax")
    print("5. random player vs alpha beta")
    print("6. Minimax vs alpha beta")
    print("7. Quit Game")
    print("\nplease pick a menu option to play against:")

    
#takes menu input
def menuInput(text): 
    return input(text)
    
def menuSystem():
    #loop that allows user to play multiple times
    while True:
        #call the menu function to display the options
        menu()
        #try-catch to catch any errors when user types in wrong format of input
        try:
            #user input is read in and displayed
            val = int(menuInput("enter your value: "))
            print("\nYou picked option:  " + str(val))
            #calls the play_game() function to allow user to play against different tic-tac-toe alogithmn
            if val == 1:
                play_game(TicTacToe(), dict(X=user, O=player(minimax_search)), verbose=True).utility
     
            elif val == 2:
                play_game(TicTacToe(), dict(X=user, O=player(alphabeta_search)), verbose=True).utility
      
            elif val == 3:   
                play_game(TicTacToe(), dict(X=user, O=player(random_player)), verbose=True).utility
 
            #Shows the execution time when 2 tic-tac-toe algorithm plays against each other
            elif val == 4:
                start_time = time.time()
                play_game(TicTacToe(), dict(X=random_player, O=player(minimax_search)), verbose=True).utility
                executon_time = (time.time() - start_time)
                print("--- Execution time: %s seconds ---" % '{0:.3f}'.format(executon_time))    
      
            elif val == 5:
                start_time = time.time()
                play_game(TicTacToe(), dict(X=random_player, O=player(alphabeta_search)), verbose=True).utility
                executon_time = (time.time() - start_time)
                print("--- Execution time: %s seconds ---" % '{0:.3f}'.format(executon_time))
        
            elif val == 6:
                start_time = time.time()
                play_game(TicTacToe(), dict(X=minimax_search, O=player(alphabeta_search)), verbose=True).utility
                executon_time = (time.time() - start_time)
                print("--- Execution time: %s seconds ---" % '{0:.3f}'.format(executon_time))
         
            #quits the menu system
            elif val == 7:
                print("\nGoodbye!!!")
                return None
            #displays valid statement when user inputs number that do not exist in menu
            else:
                print("\nUnvalid Number Try again\n")
        #catches error when user types in strings and not numbers     
        except ValueError:
            print("\nyou enterd a incorrect format\n")
  


# Menu System Displayed: Allows User to play the menu system game by calling the menuSystem() function. The expected output is a user option 1-7. The expected output is that it will allow the user to play against an algorithm or see how each alogithm performs against each other. 

In [ ]:
menuSystem()

## Unit and Integration Test. test_menu() checks if a user input is correct. test_user checks if user move is recorded from possible moves

In [ ]:
import unittest 
from io import StringIO
from unittest.mock import patch

class TestSum(unittest.TestCase):


    def test_menu(self):
        result = menuInput(6)
        self.assertEqual(result, '6')
        
        
    def test_user(self):
        userMove = user(TicTacToe(), Board())
        self.assertEqual(userMove, (0, 1))
        
    
if __name__=='__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)